In [35]:
from pprint import pprint
from nltk.tokenize import WordPunctTokenizer
from nltk import sent_tokenize,word_tokenize, RegexpParser, ne_chunk
import re

In [140]:
def train():
    from phrasematcher import PhraseMatcher
    import re

    person = PhraseMatcher('model_person', pattern_file='person.txt')
    area = PhraseMatcher('model_area', pattern_file='area.txt')
    org = PhraseMatcher('model_org', pattern_file='org.txt')
    
def entity_manual_polos(text):
    from phrasematcher import PhraseMatcher
    import re

    person = PhraseMatcher('model_person')
    area = PhraseMatcher('model_area')
    org = PhraseMatcher('model_org')

    text = text.replace(',',' ,').replace('.',' . ').replace('(',' ( ').replace(')',' ) ')
    sent_ori = text
    
    word_punct_tokenizer = WordPunctTokenizer()
    sent = word_punct_tokenizer.tokenize(sent_ori.replace('\n',' '))

    en = []
    label = []
    all_en = []
    for match in person.match(text):
        for m in match:
            all_en.append(m)
            
    enti = list(set(all_en))
    enti = sorted(enti, key=lambda x: len(x), reverse=True)
    
    for m in enti:
        all_en.append((m,'PERSON'))
        en.append(m)
        label.append('PERSON')
    
    all_en = []
    for match in org.match(text):
        for m in match:
            all_en.append(m)
            
    enti = list(set(all_en))
    enti = sorted(enti, key=lambda x: len(x), reverse=True)
    
    for m in enti:
        all_en.append((m,'ORG'))
        en.append(m)
        label.append('ORG')
    
    all_en = []
    for match in area.match(text):
        for m in match:
            all_en.append(m)
            
    enti = list(set(all_en))
    enti = sorted(enti, key=lambda x: len(x), reverse=True)
    
    for m in enti:
        all_en.append((m,'AREA'))
        en.append(m)
        label.append('AREA')
            
    final = []
    for s in sent:
        if(s in en):
            final.append((s,label[en.index(s)]))
        else:
            final.append((s,'OTH'))
    return final

def entity_IOB(tagger):
    grammar = """
    ORGANIZATION : {<ORG>+}
    PERSON : {<PERSON>+}
    AREA : {<AREA>+}
    """
    result = []
    chunkParser = RegexpParser(grammar)
    tree = chunkParser.parse(tagger)
    
    for subtree in tree.subtrees():
        if(subtree.label()=="ORGANIZATION"):
            tampung_entity = []
            for se in subtree.leaves():
                tampung_entity.append(se[0])
            entity_input = ' '.join(tampung_entity)
            result.append((entity_input,'ORGANIZATION'))
        if(subtree.label()=="PERSON"):
            tampung_entity = []
            for se in subtree.leaves():
                tampung_entity.append(se[0])
            entity_input = ' '.join(tampung_entity)
            result.append((entity_input,'PERSON'))
        if(subtree.label()=="AREA"):
            tampung_entity = []
            for se in subtree.leaves():
                tampung_entity.append(se[0])
            entity_input = ' '.join(tampung_entity)
            result.append((entity_input,'AREA'))
    return result

def generate_html_mark(sentence,entity):
    ent = []
    label = []
    for text,tipe in entity:
        ent.append(text)
        label.append(tipe)

    ent_sort = list(set(ent))
    ent_sort = sorted(ent_sort, key=lambda x: len(x), reverse=True)

    for ents in ent_sort:
        tp = label[ent.index(ents)] 
        if(tp == 'ORGANIZATION'):
            tp = 'org'
        if(tp == 'AREA'):
            tp = 'loc'
        if(tp == 'PERSON'):
            tp = 'person'
            
        if(ents.lower() != ents):
            mark_ent = '''<mark data-entity="%s">%s</mark>''' % (tp,ents)
            sentence = re.sub('''(%s)(?![^<]*>|[^<>]*<\/)''' % ents,mark_ent, sentence.rstrip())
        
    return sentence

def get_final_result(text,training = False):
    if(training):
        train()
        
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = " ".join(word_tokenize(text)).replace('.',' . ').replace(',',' . ')

    html = []
    for sent in sent_tokenize(text):
        sent_ori = sent.strip()
        entity_iob = entity_manual_polos(sent_ori)
        eIOB = entity_IOB(entity_iob)
        generate_html = generate_html_mark(sent_ori,eIOB)
        html.append(generate_html)

    html = " ".join(html)
    return html

In [137]:
def show_html(html):
    css = """
    <style>
    .entities {
            line-height: 2; 
        }

        [data-entity] {
            padding: 0.25em 0.35em;
            margin: 0px 0.25em;
            line-height: 1;
            display: inline-block;
            border-radius: 0.25em;
            border: 1px solid; 
        }

        [data-entity]::after {
            box-sizing: border-box;
            content: attr(data-entity);
            font-size: 0.6em;
            line-height: 1;
            padding: 0.35em;
            border-radius: 0.35em;
            text-transform: uppercase;
            display: inline-block;
            vertical-align: middle;
            margin: 0px 0px 0.1rem 0.5rem; 
        }

        [data-entity][data-entity="person"] {
            background: rgba(166, 226, 45, 0.2);
            border-color: rgb(166, 226, 45); 
        }

        [data-entity][data-entity="person"]::after {
            background: rgb(166, 226, 45); 
        }

        [data-entity][data-entity="norp"] {
            background: rgba(224, 0, 132, 0.2);
            border-color: rgb(224, 0, 132); 
        }

        [data-entity][data-entity="norp"]::after {
            background: rgb(224, 0, 132); 
        }

        [data-entity][data-entity="facility"] {
            background: rgba(67, 198, 252, 0.2);
            border-color: rgb(67, 198, 252); 
        }

        [data-entity][data-entity="facility"]::after {
            background: rgb(67, 198, 252); 
        }

        [data-entity][data-entity="org"] {
            background: rgba(67, 198, 252, 0.2);
            border-color: rgb(67, 198, 252); 
        }

        [data-entity][data-entity="org"]::after {
            background: rgb(67, 198, 252); 
        }

        [data-entity][data-entity="gpe"] {
            background: rgba(253, 151, 32, 0.2);
            border-color: rgb(253, 151, 32); 
        }

        [data-entity][data-entity="gpe"]::after {
            background: rgb(253, 151, 32); 
        }

        [data-entity][data-entity="loc"] {
            background: rgba(253, 151, 32, 0.2);
            border-color: rgb(253, 151, 32); 
        }

        [data-entity][data-entity="loc"]::after {
            background: rgb(253, 151, 32); 
        }

        [data-entity][data-entity="product"] {
            background: rgba(142, 125, 255, 0.2);
            border-color: rgb(142, 125, 255); 
        }

        [data-entity][data-entity="product"]::after {
            background: rgb(142, 125, 255); 
        }

        [data-entity][data-entity="event"] {
            background: rgba(255, 204, 0, 0.2);
            border-color: rgb(255, 204, 0); 
        }

        [data-entity][data-entity="event"]::after {
            background: rgb(255, 204, 0); 
        }

        [data-entity][data-entity="work_of_art"] {
            background: rgba(255, 204, 0, 0.2);
            border-color: rgb(255, 204, 0); 
        }

        [data-entity][data-entity="work_of_art"]::after {
            background: rgb(255, 204, 0); 
        }

        [data-entity][data-entity="language"] {
            background: rgba(255, 204, 0, 0.2);
            border-color: rgb(255, 204, 0); 
        }

        [data-entity][data-entity="language"]::after {
            background: rgb(255, 204, 0); 
        }

        [data-entity][data-entity="date"] {
            background: rgba(47, 187, 171, 0.2);
            border-color: rgb(47, 187, 171); 
        }

        [data-entity][data-entity="date"]::after {
            background: rgb(47, 187, 171); 
        }

        [data-entity][data-entity="time"] {
            background: rgba(47, 187, 171, 0.2);
            border-color: rgb(47, 187, 171); 
        }

        [data-entity][data-entity="time"]::after {
            background: rgb(47, 187, 171); 
        }

        [data-entity][data-entity="percent"] {
            background: rgba(153, 153, 153, 0.2);
            border-color: rgb(153, 153, 153); 
        }

        [data-entity][data-entity="percent"]::after {
            background: rgb(153, 153, 153); 
        }

        [data-entity][data-entity="money"] {
            background: rgba(153, 153, 153, 0.2);
            border-color: rgb(153, 153, 153); 
        }

        [data-entity][data-entity="money"]::after {
            background: rgb(153, 153, 153); 
        }

        [data-entity][data-entity="quantity"] {
            background: rgba(153, 153, 153, 0.2);
            border-color: rgb(153, 153, 153); 
        }

        [data-entity][data-entity="quantity"]::after {
            background: rgb(153, 153, 153); 
        }

        [data-entity][data-entity="ordinal"] {
            background: rgba(153, 153, 153, 0.2);
            border-color: rgb(153, 153, 153); 
        }

        [data-entity][data-entity="ordinal"]::after {
            background: rgb(153, 153, 153); 
        }

        [data-entity][data-entity="cardinal"] {
            background: rgba(153, 153, 153, 0.2);
            border-color: rgb(153, 153, 153); 
        }

        [data-entity][data-entity="cardinal"]::after {
            background: rgb(153, 153, 153); 
        }

        [data-entity][data-entity="entity"] {
            background: #EEE;
            border-color: #a0a0a0;
        }

        [data-entity][data-entity="entity"]::after {
            background: #a0a0a0;
        }
    </style>
    """
    from IPython.core.display import display, HTML
    display(HTML("<h2>Hasil : </h2>"))
    display(HTML(css))
    display(HTML("<p>"))
    display(HTML(html))
    display(HTML("</p>"))

In [161]:
text = '''
Partai Keadilan Sejahtera (PKS) menargetkan kemenangan 57% dari total 171 pilkada pada 2018. 
Prioritas utama PKS adalah Pilkada di Provinsi Jawa Barat.
"Tentu kami mengharapkan target terbaik menang di Jabar," kata Wakil Sekretaris Jenderal PKS Mardani Ali Sera kepada Metrotvnews.com, 
Selasa 11 Juli 2017.Yang paling sering disebut akan diusung PKS adalah Netty Heryawan, 
istri Gubernur Jawa Barat Ahmad Heryawan."Nama Deddy Mizwar yang paling mengemuka. 
Ada Ahmad Syaikhu sesuai rekomendasi dari DPW (PKS Jawa Barat)," jelas Mardani.
Deddy bukan kader PKS. Ia lebih populer sebagai aktor dan mendampingi Ahmad Heryawan memimpin Jawa Barat sejak 2013. Sedangkan Syaikhu menjabat Wakil Wali Kota Bekasi sejak 2013.Dari 
nama-nama yang muncul, PKS akan mengevaluasi melalui survei internal. 
Nama-nama tersebut juga akan dipertimbangkan dalam komunikasi partai koalisi nanti.
Komisi Pemilihan Umum (KPU) menjadwalkan tahapan pemilihan kepala daerah (pilkada) 
serentak 2018 dimulai Oktober tahun ini. Ada 171 pilkada yang pemungutan suaranya pada 27 Juni 2018.
Sosok Abdullah Gymnastiar atau Aa Gym berpotensi menjadi pesaing dalam Pemilihan 
Gubernur Jawa Barat (Pilgub Jabar) 2018.Partai NasDem memberikan keleluasaan bagi 
Ridwan Kamil untuk menjalin komunikasi dengan partai lain. Bahkan NasDem siap memfasilitasi 
Dengan hadirnya nama Pimpinan Pondok Pesantren Daarut Tauhiid Bandung sebagai salah satu kandidat 
Cagub Jawa Barat 2018 juga memberikan… Partai Golkar bertekad untuk memenangkan pemilihan 
Gubernur Jawa Barat dengan mengusung Bupati Purwakarta Dedi Mulyadi sebagai calon…
Elektabilitas Ridwan Kamil sebagai Cagub Jabar saat ini terus naik. 
Sebagai partai pengusung, Partai NasDem berusaha akan menjaga…
Lembaga survey Poltracking menilai hasil survey Cagub Jabar yang ada sekarang sangat bisa 
berubah karena masih ada 70% swing voter.…Dari 25 nama yang mencuat sebagai calon Gubernur Jabar 2018, 
elektabilitas Ridwan Kamil terus naik. Namun nama Ridwan Kamil…Keluarga yang sehat, 
anak yang sehat, menjadi hal paling penting dan berharga untuk mencapai kehidupan…
Mardani memberi sinyal jika kemesraan dengan Partai Gerindra akan tetap berlanjut, 
sambil terus berkomunikasi dengan Partai lainnya.Partai Kebangkitan Bangsa (PKB) mulai membuka pendaftaran calon kepala daerah untuk Pilkada serentak 2018.
Jika peristiwa demokrasi itu gagal dikelola dengan baik, lukanya akan mewariskan polarisasi sosial mendalam.
'''
html = get_final_result(text)
show_html(html)

In [162]:
text = '''
Daerah Nagreg di Bandung menjadi salah satu daerah yang macetnya cukup parah yang dilewati pemudik tahun ini. Mengantisipasi hal ini, pemerintah akan membangun jalan alternatif sekitar 30 hingga 40 km. Jalur ini akan melewati Ciawi-Cikijing lalu masuk ke Jalan Lingkar Jatigede, Sumedang dan Bandung sehingga bisa menghindari daerah Nagreg yang kerap menjadi titik kemacetan.Menteri Pekerjaan Umum dan Perumahan Rakyat (PUPR), Basuki Hadimuljono mengungkapkan, pada Agustus mendatang pekerjaan jalan harus sudah dilakukan mengingat proses tender untuk kontraktor telah dibuka beberapa waktu lalu."Nagreg itu kan daerah macet. Kami sudah punya program, sudah ditender, Ciawi-Cikijing lalu masuk Jalan Lingkar Jatigede-Sumedang-Bandung," kata Basuki saat ditemui di Gedung DPR RI, Jakarta, Senin (10/7/2017)."Jadi ada jalur alternatif menghindari Nagreg, itu dari Cikijing kebetulan kita harus merelokasi jalan di Jatigede karena terendam. Kalau itu enggak direlokasi, Jatigede enggak bisa maksimum. Jadi itu harus direlokasi supaya ketinggian maksimum mencapai 900 juta m3. Sambil kita merelokasi, itu kita fungsikan sebagai jalan alternatif, dari Cikijing-Sumedang-Bandung," sambungnya.Menurutnya, hal ini menjadi solusi jangka pendek persoalan macet di jalur Selatan, menunggu pembangunan jalan tol baru yang saat ini masih dalam kajian. Ditargetkan, tahun depan jalan alternatif tersebut bakal bisa dilalui oleh pemudik dengan kondisi jalan berupa rigid pavement atau lapis beton.di Nagreg bisa kita atasi. Karena kalau nunggu tol ke Tasik itu jangka panjangnya. Tapi jangka pendeknya untuk mudik, ini jalan Cikijing-Sumedang-Bandung," tukasnya.Dubes RI untuk Polandia, Peter F Gontha, menceritakan kisahnya sebagai profesional membantu orang-orang besar mengembangkan bisnis mereka.Bank Syariah Mandiri (BSM) menandatangani akad pembiayaan line facility dengan PT Federal International Finance (FIFGROUP).Menteri ESDM, Ignasius Jonan, meminta pengurangan anggaran Kementerian ESDM dari Rp 7,027 miliar dalam APBN 2017 menjadi Rp 6,56 triliun.Usai melakukan rapat kurang lebih dari 4 jam, akhirnya Komisi XI DPR memberikan persetujuan terkait dengan usulan baru asumsi dasar ekonomi pada RAPBN-P 2017.Utang pemerintah terus bertambah, hingga mencapai Rp 3.672,33 triliun. Pemerintah harus gunakan secara produktif agar tak jadi beban ke anak cucu di masa depan.INDEF (Institute for Development of Economics and Finance) menyatakan daya beli masyarakat selama Lebaran turun, kok bisa?Pemerintah tidak akan memberatkan dana Anggaran Pendapatan dan Belanja Negara (APBN) dalam merealisasikan pemindahan ibu kota negara ke luar Pulau Jawa.Pemerintah mulai menggelar rapat membahas rencana pemindahan ibu kota ke luar Jawa. Rapat digelar di Kantor Bappenas.Pemindahan ibu kota memang masih dalam kajian. Tapi bila terealisasi, bagaimana nasib Jakarta nanti?Komisi VII DPR menggelar rapat 6 jam bersama Menteri ESDM, Ignasius Jonan. Hasilnya, DPR setuju tambahan subsidi listrik Rp 7 triliun.
'''
html = get_final_result(asus)
show_html(html)

In [163]:
text = '''Pemerintah memiliki kesulitan dalam mencari investor yang ingin membangun SPBU di luar Jawa. Dengan adanya keadaan itu, maka bisa menyulitkan program pemerintah dalam menerapkan BBM satu harga di seluruh Indonesia."‎Kendala kita, kita perlu cari siapa pelaksana-pelaksana nanti, katakanlah orang yang bersedia membangun SPBU, kemudian dengan volume yang demikian kecil itu, mungkin masih kurang menarik ya," kata Manager Retail PSO PT Pertamina (Persero), Boy J. Lapian, ditemui di kantor BPH Migas, Jakarta, Senin 10 Juli 2017.Meski demikian, bilang dia, program terbaru pemerintah tersebut telah dijalankan di 20 titik di wilayah terluar Indonesia sepanjang tahun ini.‎ Program itu pun sudah merambah ke daerah Sulawesi."Kalau Papua sudah, sudah 20 titik. Sudah sekarang mulai merambah sampai ke daerah Sulawesi, terakhir di Moswaren, Sorong Selatan," jelas dia.Hingga 2019, lanjut dia, Pert‎amina membidik pelaksanaan program BBM satu harga hingga 150 wilayah. Di mana, tahun ini ditargetkan bisa terealisasi di 50 wilayah yang ada."Kendala kita mesti cari orang yang bersedia untuk berusaha disitu, tapi kita akan terus berusaha agar itu tercapai," tegas dia.Ketika itu terlaksana, dia menambahkan, maka program BBM satu harga yang telah ditetapkan pemerintah bisa terealisasi.PT Pertamina (Persero) memprioritaskan pembangunan Terminal Bahan Bakar Minyak (TBBM) baru untuk pendukung TBBM Plumpang berlokasi…PT Pertamina (Persero) mencatat konsumsi Solar mengalami penurunan 20 persen selama musim mudik dan arus balik.PT Pertamina (Persero) membidik dua daerah yang akan dibangun Terminal Bahan bakar Minyak (TBBM) baru.Pemerintah menetapkan tidak mengubah harga Bahan Bakar Minyak (BBM) jenis premium penugasan dan BBM jenis solar bersubsidi per 1 J…Sekretaris Komisi III DPRD Kalimantan Selatan H Riswandi mengapresiasi PT Pertamina (Persero) dan pihak terkait karena persediaan …PT Pertamina (Persero) memberlakukan waktu operasional 24 jam untuk Terminal Bahan Bakar Minyak (TBBM) Panjang di Lampung selama m…Pertamina menempatkan satgas khusus di titik rawan kemacetan hingga 11 Juli 2017 untuk menjaga ketersediaan bahan bakar minyak (BB…Kementerian Energi dan Sumber Daya Mineral (ESDM) mengimbau pemudik mengisi penuh tangki bahan bakar minyak (BBM) kendaraan mereka…Pertamina menegaskan harga jual bahan bakar minyak (BBM) di Kios Kemasan (KiosK), Mobile Fuel Dispenser, dan motor suplai BBM adal…Keluarga yang sehat, anak yang sehat, menjadi hal paling penting dan berharga untuk mencapai kehidup…Menteri Energi dan Sumber Daya Mineral (ESDM) Ignasius Jonan meminta tambahan subsidi listrik sebesar Rp7 triliun dalam Rancangan …Selama musim mudik Idulfitri 2017, Badan Pengatur Hilir Minyak dan Gas Bumi (BPH Migas) menyatakan, penyediaan bahan bakar minyak …Kementerian Energi dan Sumber Daya Mineral (ESDM) mempertahankan harga minyak Indonesia (Indonesia crude price/ICP) sebesar …Menteri Energi dan Sumber Daya Mineral (ESDM) Ignasius Jonan pada 22 Juni 2017 menetapkan Keputusan Menteri Nomor 2371/K/12/MEM/20…DPR menggandeng Australia dan New Zealand untuk mengembangkan potensi energi panas bumi di Indonesia.Anggota DPR RI  Haerudin meminta pemerintah meninjau ulang pencabutan subsidi listrik bagi masyarakat.PT Perusahaan Listrik Negara (PLN) (Persero) mencatat proyek pembangkit listrik 35.000 Megawatt (MW) terus berjalan sesuai rencana…Pemerintah tengah menyusun rancangan Peraturan Pemerintah (PP) tentang Perpajakan menyusul penerapan bagi hasil migas dengan skema…PT PLN (Persero) Distribusi Jateng-DIY terus menggenjot jumlah pelanggan listrik Pra Bayar di wilayahnya.Tiga wilayah distrik di kawasan Teluk Cenderawasih dan Pegunungan Tengah masuk program pelaksanaan pemberlakuan bahan bakar minyak…'''
html = get_final_result(text)
show_html(html)

In [164]:
text = '''Qatar National Bank Q.P.S.C (QNB Group) menyiapkan dana Rp 2,18 triliun untuk Bank QNB Indonesia. Dana tersebut dialokasikan sebagai dana setoran modal serta bertindak sebagai pembeli siaga dalam Rights Issue V PT Bank QNB Indonesia Tbk.Dana Rp 2,18 triliun akan digunakan Bank QNB Indonesia Tbk untuk memperluas bisnisnya di Indonesia. Tambahan modal tersebut juga merupakan komitmen QNB Group untuk senantiasa menjaga stuktur permodalan serta stabilitas pertumbuhan usaha.Dalam keterangannya, Senin (10/7), dari total dana Rp 2,18 triliun, sebesar Rp 2,06 triliun akan digunakan untuk membeli HMETD yang ditawarkan dalam Rights Issue V. Sedangkan, sisanya akan digunakan dalam Rights Issue tahun depan."Dana dari Rights Issue V, setelah dikurangi biaya-biaya emisi akan digunakan oleh bank untuk meningkatkan aset produktif, khususnya dalam bentuk penyaluran kredit," tulisnya.Dengan dilaksanakannya Rights Issue V dan tambahan setoran modal dari QNB Group, maka rasio KPMM akan berada di kisaran 16 persen, jauh di atas ketentuan minimum OJK.Hari ini, Bank QNB Indonesia juga menyelenggarakan Rapat Umum Pemegang Saham Luar Biasa (RUPSLB). RUPSLB tersebut memutuskan pengangkatan Heba Ali Ghaith Al-Tamimi dan Stephen Holden sebagai Komisaris; dan Adhiputra Tanoyo sebagai Direktur.RUPSLB juga menyetujui pengunduran diri Grant Eric Lowen sebagai Komisaris. Sehingga susunan anggota Direksi dan Dewan Komisaris Bank menjadi sebagai berikut:Heba merupakan General Manager Group Retail Banking QNB Group sekaligus menjabat beberapa posisi strategis, seperti anggota Direksi di QNB Financial Services, Doha - Qatar, anggota Direksi dan Komite Audit QNB Al Ahli, Kairo - Mesir serta anggota Direksi dan Komite Investasi Growthgate Capital, Dubai – UAE.Sementara, Stephen Holden juga memiliki pengalaman lebih dari 33 tahun di dunia perbankan Internasional. Saat ini dia menjabat sebagai General Manager di QNB Singapura.Adhiputra Tanoyo sebelumnya merupakan Direktur Bank Commonwealth Indonesia dan memiliki pengalaman lebih dari 20 tahun di bidang manajemen risiko ditunjuk sebagai Direktur untuk semakin memperkuat implementasi manajemen risiko Bank.'''
html = get_final_result(text)
show_html(html)

In [165]:
text = '''Kedatangan para sopir Grab ini ingin meminta penjelasan terkait cara dan sistem kerja yang kerap diubah-ubah oleh pihak manajemen.Sejumlah sopir Grab Medan saat melakukan protes di depan kantornya di Komplek Central Business District (CBD) Blok CC No28/29, Medan Polonia. Sopir Grab merasa ditipu oleh manajemen, Senin (10/7/2017). TRIBUN MEDAN/ARRAY A ARGUS- Merasa dibohongi oleh pihak manajemen, puluhan sopir angkutan berbasis online Grab yang ada di Medan mendadak mengepung kantornya di Komplek Central Business District (CBD) Blok CC No28/29, Medan Polonia, Senin (10/7/2017) siang.Kedatangan para sopir Grab ini ingin meminta penjelasan terkait cara dan sistem kerja yang kerap diubah-ubah oleh pihak manajemen."Ada beberapa keluhan yang kami rasa sangat merugikan para sopir. Pertama, menyangkut skema kerja," ungkap Koordinator Asosiasi Driver Online (ADO), Pandapotan kepadaPria berkaos putih ini mengatakan, pada awal perjanjian kerja, para sopir diminta mengejar 90 trip selama satu minggu. Nantinya, apabila 90 trip ini terpenuhi, maka para sopir akan mendapatkan bonus."90 trip selama satu minggu itu, sehari narik kita harus bisa dapat 13 sampai 15 trip. Belakangan, sistem ini diubah-ubah secara sepihak oleh manajemen," ungkap Pandapotan.Saat mengubah skema kerja, kata diaa, para sopir tidak pernah diajak berdialog. Bahkan yang lebih miris, skema kerja kerap diubah pada hari Senin, sehingga para sopir kelimpungan menghadapi cara kerja yang berubah-ubah itu."Perubahan sistem kerja bukannya diinformasikan pada Minggu pagi ataupun sore, kadang kala perubahan sistem kerja dibroadcast begitu saja pada malam hari," ungkap Pandapotan.Karena merasa dibohongi dan dijadikan budak oleh pihak perusahaan, sejumlah sopir sempat menyampaikan keluhan pada pimpinan Grab yang ada di Jakarta.Namun, bukannya malah mendapat solusi, pimpinan Grab di Jakarta justru buang badan pada manajemen yang ada di Medan.dan beberapa rekan media lainnya sempat meminta izin sejumlah bodyguard untuk menemui Humas Grab Medan bernama Aditya Pranata untuk mengkonfirmasikan terkait keluhan driver Grab ini.'''
html = get_final_result(text)
show_html(html)

In [166]:
text = '''Tim gabungan melakukan proses evakuasi serpihan helikopter Basarnas yang mengalami kecelakaan di gunung Butak Desa Canggal, Candiroto,Temanggung, Jateng, Rabu (5/7). - ANTARA/Anis Efizudin, JAKARTA-- PT Taspen telah melakukan pembayaran klaim atas peristiwa jatuhnya helikoter Basarnas di Gunung Butak, Temanggung awal bulan ini.Dari delapan korban jiwa, empat diantaranya adalah peserta Taspen dari Tim SAR Semarang. Direktur Perencanaan Pengembangan Bisnis dan TI Taspen Faisal Rachman bersama Gubernur Jawa Tengah Ganjar Pranowo menyerahkan santunan kepada ahli waris korban kecelakaan, yakni Alm Maulana Effendi, Catur Bambang Sulistyo, Nyoto Purwanto, dan Budi Restiyanto."Semoga lmarhum Husnul Khotimah dan keluarga yang ditinggalkan diberi kekuatan dan keikhlasan," kata Faisal dalam keterangan tertulis, Senin (10/7/2017).Sejak disahkannya Peraturan Pemerintah nomor 70/2015, tentang Jaminan Kecelakaan Kerja dan Jaminan Kematian bagi Aparatur Sipil Negara, keempat ahli waris peserta Taspen yang menjadi korban kecelakaan berhak atas sejumlah manfaat THT.Manfaat yang dimaksud tersebut antara asuransi dwiguna dan asuransi kematian serta santunan kematian, uang duka tewas, biaya pemakaman, dan beasiswa.Adapun kecelakaan terjadi pada perjalanan untuk melakukan evakuasi dan pemantauan pasca erupsi kawah Sileri, Dieng.Wacana memperbesar batasan defisit kembali mengemuka di DPR. Anggota Komisi XI DPR Andreas Eddy Susetyo mengatakan, perbesaran batasan defisit hingga 5% dipandang cukup strategis…Pengusaha asal Korea Selatan yang tergabung Asosiasi Importir Korea (KOIMA) berencana menyambangi Indonesia untuk melihat langsung dan memborong produk-produk Tanah Air.Meski perekonomian dunia mulai membaik, tetapi pemerintah tetap waspada dalam memperhatikan sejumlah faktor eksternal yang menjadi tantangan tersendiri dalam prospek ekonomi…Menteri Desa, Pembangunan Daerah Tertinggal, dan Transmigrasi Eko Putro Sandjojo memastikan pengawasan terhadap penggunaan dana desa dilakukan secara ketat.Pengenaan Pajak Pertambahan Nilai (PPN) yang akan diterapkan pemerintah ditakutkan akan membuat pemain besar beralih menjadi importir karena harga tebu atau gula menjadi…Transparansi masih menjadi persoalan pengelolaan dana desa yang implementasinya sudah memasuki tahun kedua. Badan Pameriksa Keuangan atau BPK pun didorong melakukan pengawasan…Asosiasi Asuransi Umum Indonesia (AAUI) menggelar inagurasi pengurus pusat baru periode bakti 2017-2020 pada Senin (10/7). Dalam inagurasi tersebut Ketua AAUI Dadang Sukresna…Hasil Survei Kegiatan Dunia Usaha (SKDU) mengindikasikan kegiatan usaha pada kuartal II/2017 tumbuh lebih tinggi dibandingkan dengan kuartal sebelumnya seiring dengan perbaikan…Bea Cukai berkomitmen untuk dapat menjadi institusi kepabeanan dan cukai yang kredibel dan bisa menjadi katalisator kegiatan ekonomi Indonesia dengan bekerja di lingkungan…Sejalan dengan upaya pemerintah untuk meningkatkan penerimaan negara dengan kegiatan yang lebih fair dan taat hukum, sejak Desember 2016, Bea Cukai memulai program Penguatan…Pemerintah diperkirakan akan kembali melakukan kebijakan penghematan belanja jelang akhir tahun mengingat angka defisit dalam APBN-P 2017 cukup berat.Tiga negara yang selama ini banyak menampung rekening milik Warga Negara Indonesia (WNI) yakni Singapura, Hongkong dan Switzerland, menyatakan kesiapannya untuk mematuhi…Pemerintah akan melakukan efisiensi belanja di semester II/2017, keputusan tersebut dinilai tepat mengingat efisiensi tersebut diharapkan dapat menjaga defisit anggaran yang…Badan Pemeriksa Keuangan (BPK) tengah mencari formulasi untuk masuk dan melakukan audit terhadap pengelolaan dana desa.PT Bank Pembangunan Daerah Jawa Barat dan Banten Tbk. atau BJB memastikan akan menerbitkan obligasi berkelanjutan senilai Rp2,5 triliun pada kuartal IV/2017.Otoritas Jasa Keuangan mencatat perkembangan industri Bank Perkreditan Rakyat (BPR) pada April 2017 tumbuh positif dengan total aset sebesar Rp115,2 triliun atau meningkat10,18%…emerintah memberikan pengurangan Pajak Bumi dan Bangunan (PBB) hingga 100% bagi wajib pajak yang merugi akibat bencana alam.PT Bank QNB Indonesia Tbk. menunjuk eks direktur PT Bank Commonwealth Indonesia Adhiputra Tanoyo yang ditunjuk sebagai direktur manajemen risiko perseroan.Hasil pemantauan Tindak Lanjut Rekomendasi Hasil Pemeriksaan (TLRHP) periode 2005-2016 oleh Badan Pemeriksa Keuangan sudah didapatkan pengembalian kerugian senilai Rp70,19…'''
html = get_final_result(text)
show_html(html)

In [168]:
# train()

2017-07-12 09:44:36 [28310] [INFO] Start building vocab...
2017-07-12 09:44:36 [28310] [INFO] Vocab size: 2521
2017-07-12 09:44:36 [28310] [INFO] Start compiling patterns...
2017-07-12 09:44:36 [28310] [INFO] Processing input patterns: 0
2017-07-12 09:44:36 [28310] [INFO] Start building vocab...
2017-07-12 09:44:36 [28310] [INFO] Vocab size: 747
2017-07-12 09:44:36 [28310] [INFO] Start compiling patterns...
2017-07-12 09:44:36 [28310] [INFO] Processing input patterns: 0
2017-07-12 09:44:36 [28310] [INFO] Start building vocab...
2017-07-12 09:44:36 [28310] [INFO] Vocab size: 1361
2017-07-12 09:44:36 [28310] [INFO] Start compiling patterns...
2017-07-12 09:44:36 [28310] [INFO] Processing input patterns: 0
